In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import mlab
import scipy
import hashlib
import timeit
import cv2
import sys
import os
import librosa
import pyaudio
import wave
import pymysql
import mysql.connector
import sounddevice as sd
import librosa.display
from IPython.display import Audio
from scipy.ndimage.filters import maximum_filter

In [2]:
def spectro(arr):
    starr=librosa.stft(arr,n_fft=2048,hop_length=512)
    specmat = (np.abs(starr))**2
    return specmat

In [3]:
def peaks(starr):   
    size = (15*2) +1
    global local_max 
    local_max = maximum_filter(starr, size) == starr
    time_list = []
    freq_list = []
    L = starr.shape[0]
    B = starr.shape[1]
    for i in range(B):
        for j in range(L):
            if (local_max[j][i]): 
                
                if (starr[j][i]<=15):  
                    local_max[j][i] = False
                else:
                    time_list.append(i)
                    freq_list.append(j)
                    
                ver_bin = np.argpartition(starr[:,i], -6)[-6:]
                for k in range(len(ver_bin)):
                    if (starr[ver_bin[k]][i]>15):
                        time_list.append(i)
                        freq_list.append(ver_bin[k])
    return time_list, freq_list, L, B

In [4]:
def get_hash(time_list, freq_list, L, B,song_id):
    t = time_list
    f = freq_list
    n = len(time_list)
    hashes = []
    count = 0
    k=0
    for i in range(n):
        for j in range(0,300):
            k+=1
            if ((i+j)<n):
                freq1 = f[i]
                freq2 = f[i+j]
                time_delta = t[i+j] - t[i]
                if ( (time_delta > 0) & (time_delta < 300) ):
                    count += 1
                    h = hashlib.sha224(f"{str(freq1)}|{str(freq2)}|{time_delta}".encode('utf-8'))
                    hash1 = h.hexdigest()
                    hashes.append((song_id,hash1[0:20],t[i]))
                    if (count>=20):
                        break  
    return hashes

In [66]:
def add_songs_to_database():
    
    try:
        mydb =  mysql.connector.connect(user='root', password='kishan@2022', host='127.0.0.1',database='kishandata')
    except:
        print("Unable to conect to the database")
    
    mycursor = mydb.cursor()
    song_name = input("Enter the song name ")
    song_id=int(input("enter song id"))
    filename='song_to_add/'+song_name+'.wav'
    recarr,sr=librosa.load(filename)
    #plot(recarr,sr)
    sql = "SELECT count(*) from songs where song_name = %s and fingerprinted = %s"
    val = (song_name,"1")
    mycursor.execute(sql,val)
    arr = mycursor.fetchall()
    if (arr[0][0]==1):
        print("Song ", song_name," is already fingerprinted")
        return
    else:
        sql = "INSERT INTO songs (song_name,fingerprinted,SONG_ID) VALUES (%s,%s,%s)"
        val = (song_name,"1",song_id)
        mycursor.execute(sql,val)
        
    spec_matrix = spectro(recarr)
    print("Spectrogram successfully made")
    
    time_list, frequency_list, L, B = peaks(spec_matrix)
    print("Found ", len(time_list)," peaks")
    
    hashes = get_hash(time_list, frequency_list, L, B,song_id)
    print("Made ",len(hashes)," hashes")
    hashes = list(set(hashes))
    print("Found ",len(hashes)," unique hashes")
    
    val = hashes
    sql = "INSERT INTO fingerprints (song_id,hash,offset) VALUES (%s,%s,%s)"
    mycursor.executemany(sql,val)
    mydb.commit()
    print(mycursor.rowcount, "record(s) inserted for song ",song_name)
    
    mycursor.close()
    mydb.close()

In [6]:
def recognize():
    print("How do you want to read the song")
    print("    1. Using the microphone")
    print("    2. Reading the sample file from the system")
    choice = input("Enter the choice : ")
    if(choice == '1'):
        filename = "test" + ".wav"
        chunk = 1024
        channels = 1
        sample_rate = 44100
        record_seconds = 10
        FORMAT = pyaudio.paInt16
        p = pyaudio.PyAudio()
        stream = p.open(format=FORMAT,
                rate=sample_rate,
                input=True,
                output=True,
                channels=channels,
                frames_per_buffer=chunk)
        frames = []
        print("Recording...")        
        for i in range(int(44100 / chunk * record_seconds)):
            data = stream.read(chunk)
            frames.append(data)
        
        print("Recording ended...")
        stream.stop_stream()
        stream.close()
        p.terminate()
        wf = wave.open(filename, "wb")
        wf.setnchannels(channels)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(sample_rate)
        wf.writeframes(b"".join(frames))
        wf.close()
        path = filename 
        recarr,sampling_rate=librosa.load(path)
        recognize_the_song(recarr, sampling_rate)
       
    elif (choice == '2'):
        song_name = input("Enter the name of the song you want to recognize: ")        
        path = 'testing/' + song_name + '.wav' 
        recarr,sr=librosa.load(path)
        recognize_the_song(recarr,sr)
    else:
        print("PLease enter a valid choice number")
        
    return

In [7]:
def plot(audio,sampling_rate):
    plt.figure(figsize=(20, 10))
    librosa.display.waveshow(audio, sr=sampling_rate)
    X = librosa.stft(audio)
    Xdb = librosa.amplitude_to_db(abs(X))
    plt.figure(figsize=(20, 10))
    librosa.display.specshow(Xdb, sr=sampling_rate, x_axis='time', y_axis='log')
    plt.colorbar()

In [54]:
def recognize_the_song(audio, sampling_rate):
    try:
        mydb = mysql.connector.connect(user='root', password='kishan@2022', host='127.0.0.1',database='kishandata')
    except:
        print("Unable to conect to the database")
    
    mycursor = mydb.cursor()
   
    spec_matrix = spectro(audio)
    print("Spectrogram successfully made")
    
    time_list, frequency_list, L, B = peaks(spec_matrix)
    print("Peaks found", len(time_list))
    
    hashes = get_hash(time_list, frequency_list, L, B,song_id=101)
    print("Hashes found",len(hashes))
    hashes = list(set(hashes))
    print("Unique Hashes found",len(hashes))
    
    unique_hashes = hashes
    
    bins = [{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}]

    for sample_hash in unique_hashes:
        
        sql = ("SELECT song_id, offset from fingerprints where hash = %s")
        val = (sample_hash[1],)
        mycursor.execute(sql,val)
        id_and_offset = (mycursor.fetchall())
        sample_offset = sample_hash[2] 
        for j in id_and_offset:
            song_id = (j[0])
            database_offset = j[1]
            difference = int(database_offset) - int(sample_offset)
            bins[song_id][difference] = bins[song_id].get(difference,0) + 1
            
    print(bins)
    score = {}
    for i in range(len(bins)):
        if bins[i] != {}:
            score[i] = bins[i].get( max(bins[i], key = bins[i].get) )
    print("scores",score)
    try:
        Best_match = max(score,key = score.get)
    except:
        print("Error")
        return 
    sql = ("SELECT song_name from songs where song_id = %s")
    val = (Best_match,)
    mycursor.execute(sql,val)
    name_best_match = mycursor.fetchone()
    
    print("The best match song is    ", name_best_match[0],"    with a score of ",score[Best_match])
    
    return

In [68]:
print("    1. Add songs to the data base")
print("    2. Recognize a song")
choice = input("Enter your choice number: ")
if (choice == '1'):
    add_songs_to_database()
elif (choice =='2'):
    recognize()
else:
    print("PLease enter a valid choice number")

    1. Add songs to the data base
    2. Recognize a song
Enter your choice number: 2
How do you want to read the song
    1. Using the microphone
    2. Reading the sample file from the system
Enter the choice : 2
Enter the name of the song you want to recognize: test
Spectrogram successfully made
Peaks found 1747
Hashes found 1752
Unique Hashes found 869
[{1319: 65, 1475: 16, 130: 64, 596: 58, 234: 478, 1061: 152, 1371: 81, 647: 105, 182: 75, 751: 157, 544: 157, 854: 139, 1422: 10, 905: 16, 1215: 6, 750: 47, 1009: 93, 27: 150, 1164: 138, 1474: 56, 440: 44, 492: 58, 1267: 63, 802: 51, 285: 21, 1577: 8, 1112: 34, 595: 28, 958: 30, 832: 6, 337: 93, 1629: 5, 957: 94, 109: 4, 389: 51, 1527: 10, 1216: 48, 28: 12, 648: 26, 131: 60, 441: 84, 1268: 45, -120: 3, 707: 3, 242: 4, 1017: 3, 807: 3, -172: 4, 965: 4, 1121: 3, 293: 3, -224: 4, 729: 4, -76: 96, 906: 60, 699: 84, 1526: 12, 6: 5, 79: 37, 1372: 5, -127: 5, -283: 67, -180: 57, 857: 7, 1167: 8, 286: 57, 1093: 3, 1214: 5, 1010: 5, 284: 5, 8

In [56]:
1455 #
136
1
1611 #
136
1
266 #
136
1
732 #
136
1
370 #
136
1

1